# Análisis de datos obtenidos desde un secuenciador masivo

Estructura

1. Seteo del ambiente
 - Conda
 - Jupyter Notebook
 - Creación de estructura de directorios
 - Descarga de DNA de referencia
 - Obtención de archivos desde el secuenciador
1. Control de calidad
1. Trimming
1. Alineamiento de datos podados pareados
 - Indexación de ADN de referencia
 - Alineamiento
1. Variant calling
 - Creación de archivo pileup
 - Llamado de variantes

In [6]:
!conda config --add channels bioconda

In [7]:
!conda install -c --yes bioconda fastqc # para analizar la calidad de los reads
!conda install -c --yes bioconda bwa # para alinear los reads contrar un genoma de referencia
!conda install -c --yes bioconda samtools # para transformar el archivo de alieamiento en un formato desplegable de manera gráfica
!conda install -c --yes bioconda igv # para visualizar gráficamente los aineamientos
!conda install -c --yes bioconda bcftools # para identificar las variantes (mutaciones)
!conda install -c --yes conda-forge py-bgzip # para comprimir en DNAref previo al pileup del llamado de variantes

usage: conda install [-h] [--revision REVISION] [-n ENVIRONMENT | -p PATH]
                     [-c CHANNEL] [--use-local] [--override-channels]
                     [--repodata-fn REPODATA_FNS] [--experimental {jlap,lock}]
                     [--no-lock] [--repodata-use-zst | --no-repodata-use-zst]
                     [--strict-channel-priority] [--no-channel-priority]
                     [--no-deps | --only-deps] [--no-pin] [--copy]
                     [--no-shortcuts] [--shortcuts-only SHORTCUTS_ONLY] [-C]
                     [-k] [--offline] [--json] [-v] [-q] [-d] [-y]
                     [--download-only] [--show-channel-urls] [--file FILE]
                     [--solver {classic,libmamba}] [--force-reinstall]
                     [--freeze-installed | --update-deps | -S | --update-all | --update-specs]
                     [-m] [--clobber] [--dev]
                     [package_spec ...]
conda install: error: argument -c/--channel: expected one argument
usage: conda install

In [3]:
!mkdir DNAref
!mkdir RawReads
!mkdir FastQC_rawReads
!mkdir Trimmed_reads
!mkdir FastQC_trimmedReads
!mkdir Aligned_reads
!mkdir Variant_calling

mkdir: no se puede crear el directorio «DNAref»: El archivo ya existe
mkdir: no se puede crear el directorio «RawReads»: El archivo ya existe
mkdir: no se puede crear el directorio «Trimmed_reads»: El archivo ya existe
mkdir: no se puede crear el directorio «FastQC_trimmedReads»: El archivo ya existe
mkdir: no se puede crear el directorio «Variant_calling»: El archivo ya existe


In [9]:
%cd RawReads/
# creación de links simbólicos
!ln -s Libreria-15-82-R_S3_L001_R1_001.fastq.gz RS3L1R1.fq.gz
!ln -s Libreria-15-82-R_S3_L001_R2_001.fastq.gz RS3L1R2.fq.gz
!ln -s Libreria-15-86-R_S17_L001_R1_001.fastq.gz RS17L1R1.fq.gz
!ln -s Libreria-15-86-R_S17_L001_R2_001.fastq.gz RS17L1R2.fq.gz

/home/sebe/VIIsem/bioInfo_INFO241/Tareas/Tarea3/Tarea3G3/RawReads
ln: fallo al crear el enlace simbólico 'RS3L1R1.fq.gz': El archivo ya existe
ln: fallo al crear el enlace simbólico 'RS3L1R2.fq.gz': El archivo ya existe
ln: fallo al crear el enlace simbólico 'RS17L1R1.fq.gz': El archivo ya existe
ln: fallo al crear el enlace simbólico 'RS17L1R2.fq.gz': El archivo ya existe


In [13]:
%cd DNAref
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.40_GRCh38.p14/GCF_000001405.40_GRCh38.p14_genomic.fna.gz
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.40_GRCh38.p14/GCF_000001405.40_GRCh38.p14_genomic.gff.gz 
%cd ..

/home/sebe/VIIsem/bioInfo_INFO241/Tareas/Tarea3/Tarea3G3/DNAref
--2024-07-08 19:34:58--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.40_GRCh38.p14/GCF_000001405.40_GRCh38.p14_genomic.fna.gz
Resolviendo ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 2607:f220:41e:250::10, 2607:f220:41e:250::7, 130.14.250.11, ...
Conectando con ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)[2607:f220:41e:250::10]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 972898531 (928M) [application/x-gzip]
Guardando como: ‘GCF_000001405.40_GRCh38.p14_genomic.fna.gz’

GCF_000001405.40_GR 100%[===================>] 927,83M  8,52MB/s    en 2m 38s  

2024-07-08 19:37:36 (5,88 MB/s) - ‘GCF_000001405.40_GRCh38.p14_genomic.fna.gz’ guardado [972898531/972898531]

/home/sebe/VIIsem/bioInfo_INFO241/Tareas/Tarea3/Tarea3G3


In [ ]:
%cd DNAref
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.40_GRCh38.p14/GCF_000001405.40_GRCh38.p14_genomic.gff.gz 
%cd ..

## Secuenciación cruda paciente RS3 FASTQC

In [1]:
!fastqc RawReads/RS3L1R1.fq.gz -o FastQC_rawReads/
!fastqc RawReads/RS3L1R2.fq.gz -o FastQC_rawReads/

application/gzip
Started analysis of RS3L1R1.fq.gz
Approx 5% complete for RS3L1R1.fq.gz
Approx 10% complete for RS3L1R1.fq.gz
Approx 15% complete for RS3L1R1.fq.gz
Approx 20% complete for RS3L1R1.fq.gz
Approx 25% complete for RS3L1R1.fq.gz
Approx 30% complete for RS3L1R1.fq.gz
Approx 35% complete for RS3L1R1.fq.gz
Approx 40% complete for RS3L1R1.fq.gz
Approx 45% complete for RS3L1R1.fq.gz
Approx 50% complete for RS3L1R1.fq.gz
Approx 55% complete for RS3L1R1.fq.gz
Approx 60% complete for RS3L1R1.fq.gz
Approx 65% complete for RS3L1R1.fq.gz
Approx 70% complete for RS3L1R1.fq.gz
Approx 75% complete for RS3L1R1.fq.gz
Approx 80% complete for RS3L1R1.fq.gz
Approx 85% complete for RS3L1R1.fq.gz
Approx 90% complete for RS3L1R1.fq.gz
Approx 95% complete for RS3L1R1.fq.gz
Analysis complete for RS3L1R1.fq.gz
application/gzip
Started analysis of RS3L1R2.fq.gz
Approx 5% complete for RS3L1R2.fq.gz
Approx 10% complete for RS3L1R2.fq.gz
Approx 15% complete for RS3L1R2.fq.gz
Approx 20% complete for RS3L

## PODA 

In [4]:
!trimmomatic PE RawReads/RS3L1R1.fq.gz RawReads/RS3L1R2.fq.gz Trimmed_reads/p_RS3R1.fq.gz Trimmed_reads/u_RS3R1.fq.gz Trimmed_reads/p_RS3R2.fq.gz Trimmed_reads/u_RS3R2.fq.gz LEADING:3 TRAILING:3 SLIDINGWINDOW:4:20 MINLEN:20

TrimmomaticPE: Started with arguments:
 RawReads/RS3L1R1.fq.gz RawReads/RS3L1R2.fq.gz Trimmed_reads/p_RS3R1.fq.gz Trimmed_reads/u_RS3R1.fq.gz Trimmed_reads/p_RS3R2.fq.gz Trimmed_reads/u_RS3R2.fq.gz LEADING:3 TRAILING:3 SLIDINGWINDOW:4:20 MINLEN:20
Quality encoding detected as phred33
Input Read Pairs: 32914 Both Surviving: 25836 (78,50%) Forward Only Surviving: 6831 (20,75%) Reverse Only Surviving: 30 (0,09%) Dropped: 217 (0,66%)
TrimmomaticPE: Completed successfully


In [5]:
!trimmomatic PE RawReads/RS17L1R1.fq.gz RawReads/RS17L1R2.fq.gz Trimmed_reads/p_RS17R1.fq.gz Trimmed_reads/u_RS17R1.fq.gz Trimmed_reads/p_RS17R2.fq.gz Trimmed_reads/u_RS17R2.fq.gz LEADING:3 TRAILING:3 SLIDINGWINDOW:4:20 MINLEN:20

TrimmomaticPE: Started with arguments:
 RawReads/RS17L1R1.fq.gz RawReads/RS17L1R2.fq.gz Trimmed_reads/p_RS17R1.fq.gz Trimmed_reads/u_RS17R1.fq.gz Trimmed_reads/p_RS17R2.fq.gz Trimmed_reads/u_RS17R2.fq.gz LEADING:3 TRAILING:3 SLIDINGWINDOW:4:20 MINLEN:20
Quality encoding detected as phred33
Input Read Pairs: 41779 Both Surviving: 39354 (94,20%) Forward Only Surviving: 2042 (4,89%) Reverse Only Surviving: 222 (0,53%) Dropped: 161 (0,39%)
TrimmomaticPE: Completed successfully


In [6]:
!for i in `ls Trimmed_reads/*.fq.gz`; do fastqc $i -o FastQC_trimmedReads/; done

application/gzip
Started analysis of p_RS17R1.fq.gz
Approx 5% complete for p_RS17R1.fq.gz
Approx 10% complete for p_RS17R1.fq.gz
Approx 15% complete for p_RS17R1.fq.gz
Approx 20% complete for p_RS17R1.fq.gz
Approx 25% complete for p_RS17R1.fq.gz
Approx 30% complete for p_RS17R1.fq.gz
Approx 35% complete for p_RS17R1.fq.gz
Approx 40% complete for p_RS17R1.fq.gz
Approx 45% complete for p_RS17R1.fq.gz
Approx 50% complete for p_RS17R1.fq.gz
Approx 55% complete for p_RS17R1.fq.gz
Approx 60% complete for p_RS17R1.fq.gz
Approx 65% complete for p_RS17R1.fq.gz
Approx 70% complete for p_RS17R1.fq.gz
Approx 75% complete for p_RS17R1.fq.gz
Approx 80% complete for p_RS17R1.fq.gz
Approx 85% complete for p_RS17R1.fq.gz
Approx 90% complete for p_RS17R1.fq.gz
Approx 95% complete for p_RS17R1.fq.gz
Analysis complete for p_RS17R1.fq.gz
application/gzip
Started analysis of p_RS17R2.fq.gz
Approx 5% complete for p_RS17R2.fq.gz
Approx 10% complete for p_RS17R2.fq.gz
Approx 15% complete for p_RS17R2.fq.gz
App

indexacion de adn referencia

In [1]:
!time bwa index DNAref/GCF_000001405.40_GRCh38.p14_genomic.fna.gz

[bwa_index] Pack FASTA... 26.41 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=6596861272, availableWord=476179232
[BWTIncConstructFromPacked] 10 iterations done. 99999992 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 199999992 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 299999992 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 399999992 characters processed.
[BWTIncConstructFromPacked] 50 iterations done. 499999992 characters processed.
[BWTIncConstructFromPacked] 60 iterations done. 599999992 characters processed.
[BWTIncConstructFromPacked] 70 iterations done. 699999992 characters processed.
[BWTIncConstructFromPacked] 80 iterations done. 799999992 characters processed.
[BWTIncConstructFromPacked] 90 iterations done. 899999992 characters processed.
[BWTIncConstructFromPacked] 100 iterations done. 999999992 characters processed.
[BWTIncConstructFromPacked] 110 iterations done. 

## ALINEAMIENTO

In [5]:
!bwa mem DNAref/GCF_000001405.40_GRCh38.p14_genomic.fna.gz Trimmed_reads/p_RS3R1.fq.gz Trimmed_reads/p_RS3R2.fq.gz > Aligned_reads/trimPairedRS3.sam

[bwt_restore_sa] fail to open file 'DNAref/GCF_000001405.40_GRCh38.p14_genomic.fna.gz.sa' : No such file or directory


In [6]:
!bwa mem DNAref/GCF_000001405.40_GRCh38.p14_genomic.fna.gz Trimmed_reads/p_RS17R1.fq Trimmed_reads/p_RS17R2.fq > Aligned_reads/trimPairedRS17.sam

[bwt_restore_sa] fail to open file 'DNAref/GCF_000001405.40_GRCh38.p14_genomic.fna.gz.sa' : No such file or directory


###  Para verlo con IGV

In [ ]:
%cd Align_reads/
!samtools sort trimPaired.sam -o sorted_trimPaired.bam
!samtools index sorted_trimPaired.bam
%cd ..

In [ ]:
!igv Aligned_reads/sorted_trimPaired.bam

## Variant calling

In [ ]:
!gunzip DNAref/GCF_000001405.40_GRCh38.p14_genomic.fna.gz #descomprimir .gz
!bgzip DNAref/GCF_000001405.40_GRCh38.p14_genomic.fna # volver a comprimir el FASTA con bgzip


In [ ]:
!samtools mpileup Align_reads/sorted_trimPaired.bam -g -f DNAref/GCF_000001405.40_GRCh38.p14_genomic.fna.gz > Variant_calling/pileup.bcf